In [111]:
import torch
from torch import nn
from torch.autograd import Variable 
import re
import random
import tqdm
import time

In [112]:
!wget https://s3.amazonaws.com/text-datasets/nietzsche.txt

--2022-12-05 18:45:54--  https://s3.amazonaws.com/text-datasets/nietzsche.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.140.142, 54.231.160.136, 52.216.77.150, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.140.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 600901 (587K) [text/plain]
Saving to: ‘nietzsche.txt.2’

nietzsche.txt.2     100%[===================>] 586.82K  --.-KB/s    in 0.1s    

2022-12-05 18:45:54 (4.32 MB/s) - ‘nietzsche.txt.2’ saved [600901/600901]



In [113]:
with open('nietzsche.txt', encoding='utf-8') as f:
    text = f.read().lower()
print('length:', len(text))
text = re.sub('[^a-z ]', ' ', text)
text = re.sub('\s+', ' ', text)

length: 600893


In [114]:
text[:100]

'preface supposing that truth is a woman what then is there not ground for suspecting that all philos'

In [115]:
INDEX_TO_CHAR = sorted(list(set(text)))
CHAR_TO_INDEX = {c: i for i, c in enumerate(INDEX_TO_CHAR)}

In [116]:
CHAR_TO_INDEX

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [117]:
MAX_LEN = 40
STEP = 3
SENTENCES = []
NEXT_CHARS = []
for i in range(0, len(text) - MAX_LEN, STEP):
    SENTENCES.append(text[i: i + MAX_LEN])
    NEXT_CHARS.append(text[i + MAX_LEN])
print('Num sents:', len(SENTENCES))

Num sents: 193075


In [118]:
print('Vectorization...')
X = torch.zeros((len(SENTENCES), MAX_LEN), dtype=int)
Y = torch.zeros((len(SENTENCES)), dtype=int)
for i, sentence in enumerate(SENTENCES):
    for t, char in enumerate(sentence):
        X[i, t] = CHAR_TO_INDEX[char]
    Y[i] = CHAR_TO_INDEX[NEXT_CHARS[i]]

Vectorization...


In [119]:
X[0:1], Y[0]

(tensor([[16, 18,  5,  6,  1,  3,  5,  0, 19, 21, 16, 16, 15, 19,  9, 14,  7,  0,
          20,  8,  1, 20,  0, 20, 18, 21, 20,  8,  0,  9, 19,  0,  1,  0, 23, 15,
          13,  1, 14,  0]]), tensor(23))

In [120]:
BATCH_SIZE=512
dataset = torch.utils.data.TensorDataset(X, Y)
data = torch.utils.data.DataLoader(dataset, BATCH_SIZE, shuffle=True)


In [121]:
class NeuralNetwork(nn.Module):
    def __init__(self, rnnClass, dictionary_size, embedding_size, num_hiddens, num_classes):
        super().__init__()
        
        self.num_hiddens = num_hiddens
        self.embedding_size = embedding_size
        self.embedding = nn.Embedding(dictionary_size, embedding_size)
        self.hidden = rnnClass(embedding_size, num_hiddens, batch_first=True)
        self.output = nn.Linear(num_hiddens, num_classes)
        
    def forward(self, X):
        out = self.embedding(X)
        o, (state, _) = self.hidden(out)
        hn = state.view(-1, self.num_hiddens)
        predictions = self.output(hn)
        return predictions

In [122]:
model = NeuralNetwork(nn.LSTM, len(CHAR_TO_INDEX), 64, 128, len(CHAR_TO_INDEX))

In [123]:
model(X[0:1])

tensor([[ 0.0725, -0.1041,  0.1140,  0.0480, -0.0444,  0.0257,  0.0658, -0.1149,
         -0.1419, -0.1193, -0.1509, -0.0392, -0.1224, -0.0724,  0.0612,  0.0406,
         -0.0045, -0.1775,  0.0603, -0.0367, -0.1145, -0.0230,  0.0225,  0.1140,
         -0.1837, -0.1113, -0.0811]], grad_fn=<AddmmBackward0>)

In [124]:
model = model.cuda()

In [125]:
def sample(preds):
    softmaxed = torch.softmax(preds, 0)
    probas = torch.distributions.multinomial.Multinomial(1, softmaxed).sample()
    return probas.argmax()

def generate_text():
    start_index = random.randint(0, len(text) - MAX_LEN - 1)

    generated = ''
    sentence = text[start_index: start_index + MAX_LEN]
    generated += sentence

    for i in range(MAX_LEN):
        x_pred = torch.zeros((1, MAX_LEN), dtype=int)
        for t, char in enumerate(generated[-MAX_LEN:]):
            x_pred[0, t] = CHAR_TO_INDEX[char]

        preds = model(x_pred.cuda())[0].cpu()
        next_char = INDEX_TO_CHAR[sample(preds)]
        generated = generated + next_char

    print(generated[:MAX_LEN] + '|' + generated[MAX_LEN:])

In [126]:
generate_text()

o present long heavy difficult dangerous|oo wshlnpehrrygpjgaqgrlcdtx eegryoniukin


In [127]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [128]:
for ep in range(100):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    model.train()
    for X_b, y_b in tqdm.tqdm_notebook(data):
        X_b, y_b = X_b.cuda(), y_b.cuda()
        optimizer.zero_grad()
        answers = model.forward(X_b)
        loss = criterion(answers, y_b)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    model.eval()
    generate_text()

<ipython-input-128-894c497e4419>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for X_b, y_b in tqdm.tqdm_notebook(data):


  0%|          | 0/378 [00:00<?, ?it/s]

Epoch 0. Time: 4.801, Train loss: 2.173
tures that are destitute of ambition the| rean ittane ad condilem to is of poprna


  0%|          | 0/378 [00:00<?, ?it/s]

Epoch 1. Time: 4.780, Train loss: 1.780
an instrument we may say he is a mirror |verity of thatmentlod feay ty mangeralap


...

Epoch 97. Time: 4.884, Train loss: 0.948
 the value of the thing it fights for am|ong them as must be exception are natura


  0%|          | 0/378 [00:00<?, ?it/s]

Epoch 98. Time: 4.936, Train loss: 0.947
ork itself upwards to future supremacy a|nd absorbotions to death in which does n


  0%|          | 0/378 [00:00<?, ?it/s]

Epoch 99. Time: 4.936, Train loss: 0.944
the slaves and dependents of all sorts i|n all refinements be regarded as criter 
